In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA, TruncatedSVD, KernelPCA
from sklearn.linear_model import LinearRegression

# 1) PCA
Para esse exercício, usaremos novamente dados das eleições municipais de 2000:

In [2]:
link = 'https://raw.githubusercontent.com/FLS-6497/datasets/main/aula7/eleicoes2000.csv'
dados = pd.read_csv(link, sep=';', decimal=",").drop(['cod_mun_ibge', 'nome_municipio'], axis=1)

## a) Criando uma pipeline
Com os dados, implemente uma pipeline de classificação que use PCA para reduzir o número de features nos dados. Seu objetivo será predizer qual partido governa dado município. Calcule alguma métrica de validação.


In [3]:
dados[['outros_partidos', 'pmdb_psdb_pfl']] = pd.get_dummies(dados.partido)
dados = pd.concat([
        dados, 
        pd.get_dummies(dados.uf)
        ], 
        axis=1
    )
dados = dados.drop(['uf', 'coligacao', 'partido', 'outros_partidos'], axis=1)

y = dados.pmdb_psdb_pfl
X = dados.loc[:, dados.columns != 'pmdb_psdb_pfl']

X_train, X_test, y_train, y_test = train_test_split(X, y)

pipe = Pipeline([
    ('scale', StandardScaler()), 
    ('pca', PCA()), 
    ('svr', SVR())
    ])

pipe.fit(X_train, y_train)
mean_squared_error(y_test, pipe.predict(X_test), squared=False)

0.5315855978406862

In [10]:
pipe = Pipeline([
    ('scale', StandardScaler()), 
    ('pca', PCA()), 
    ('ols', LinearRegression())
    ])

pipe.fit(X_train, y_train)
mean_squared_error(y_test, pipe.predict(X_test), squared=False)

0.480491289319278

## b) Testando diferentes pipelines
Partindo da pipeline anterior, crie diferentes pipelines alterando o número de dimensões no PCA para 2, 3, 4 e 5. Rode essas pipelines e compare seus resultados.

In [11]:
def testing_dim_pca(dim):
    
    pipe = Pipeline([
    ('scale', StandardScaler()), 
    ('pca', PCA(n_components=dim)), 
    ('ols', LinearRegression())
    ])

    pipe.fit(X_train, y_train)
    return mean_squared_error(y_test, pipe.predict(X_test), squared=False)

dims = [2, 3, 4, 5]
[testing_dim_pca(dim=i) for i in dims]

[0.4896977495013797,
 0.48963289868914983,
 0.4900648912517327,
 0.4901177181735851]

In [12]:
def testing_dim_pca(dim):
    
    pipe = Pipeline([
    ('scale', StandardScaler()), 
    ('pca', PCA(n_components=dim)), 
    ('svr', SVR())
    ])

    pipe.fit(X_train, y_train)
    return mean_squared_error(y_test, pipe.predict(X_test), squared=False)

dims = [2, 3, 4, 5]
[testing_dim_pca(dim=i) for i in dims]

[0.5568174214748597,
 0.5477401092449328,
 0.5365067197732825,
 0.5256496643790551]

## c) Alternativas
Checando a documentação do seu framework, implemente alguma alternativa ao PCA (exemplo: kernel PCA).

In [5]:
def testing_dim_tsvd(dim):
    
    pipe = Pipeline([
    ('scale', StandardScaler()), 
    ('tsvd', TruncatedSVD(n_components=dim)), 
    ('svr', SVR())
    ])

    pipe.fit(X_train, y_train)
    return mean_squared_error(y_test, pipe.predict(X_test), squared=False)

dims = [2, 3, 4, 5]
[testing_dim_tsvd(dim=i) for i in dims]

[0.557026471976934, 0.5481071961019471, 0.5322127957447597, 0.5307991117232226]

In [8]:
def testing_dim_kpca(dim):
    
    pipe = Pipeline([
    ('scale', StandardScaler()), 
    ('kpca', KernelPCA(n_components=dim)), 
    ('svr', SVR())
    ])

    pipe.fit(X_train, y_train)
    return mean_squared_error(y_test, pipe.predict(X_test), squared=False)

dims = [2, 3, 4, 5]
[testing_dim_kpca(dim=i) for i in dims]

[0.5568619570577704, 0.548775536658938, 0.5366546794660529, 0.5243517468388741]

## 2) Clustering
Para esse exercício, usaremos uma nova base de dados com gastos reportados dos gabinetes de deputados e deputadas federais até agora, em 2022, separados por tipo de gasto:

In [13]:
link = 'https://raw.githubusercontent.com/FLS-6497/datasets/main/aula8/gabinetes22.csv'
dados = pd.read_csv(link, sep=';', decimal=",")
dados2 = dados.select_dtypes(['number'])

## a) K-means
Use K-means para separar os e as parlamentares em 4 grupos. Adicione essa classificação na base de dados original e a explore para tentar compreender quais são esses grupos.